## Network Biology 
### Coding Assignment 3
#### Submitted By: Divyanshu Srivastava

In [25]:
import networkx as nx
import numpy as np
import matplotlib as plt
import random

#### Question 1


In [410]:
## Initialization

genes = 25
tfbs_pool = 10
tfbs_per_gene = 3

gene_pool = {}
for g in range(genes):
    gene_pool[g] = random.sample(range(tfbs_pool), tfbs_per_gene)

steps_of_evolution = 100

p_gene_dublication = 0.25
p_gene_deletion = 0.25
p_tfbs_dublication = 0.25
p_tfbs_deletion = 0.25
p_vector = [p_gene_dublication, p_gene_deletion, p_tfbs_dublication, p_tfbs_deletion]



## Evolution
for s in range(steps_of_evolution):
    r = np.random.choice(len(p_vector), p = p_vector)
    if r == 0:
#         print "Evolution Step : " + str (s) + " Gene Dublication"
        gene_to_dublicate = random.sample(range(genes), 1)[0]
        gene_pool[genes] = gene_pool[gene_to_dublicate]
        genes = genes + 1;
    elif r == 1:
#         print "Evolution Step : " + str (s) + " Gene Deletion"
        gene_to_delete = random.sample(range(genes), 1)[0]
        gene_pool.pop(gene_to_delete)
        for i in range(gene_to_delete, genes-1):
            gene_pool[i] = gene_pool.get(i+1)
        if not gene_to_delete == genes - 1:
            gene_pool.pop(genes - 1)
        genes = genes-1
        if genes == 0:
            print "Gene Pool Empty !"
            break
    elif r == 2:
#         print "Evolution Step : " + str (s) + " TFBS Dublication"
        tfbs_probability = np.array(range(0, tfbs_pool))
        for g in gene_pool:
            for value in gene_pool[g]:
                tfbs_probability[value] = tfbs_probability[value]+1
        tfbs_probability = tfbs_probability.astype(np.float)
        tfbs_probability = tfbs_probability / np.sum(tfbs_probability)
        tfbs_to_dublicate = np.random.choice(tfbs_pool, p = tfbs_probability)
        flag = False
        while not flag:
            gene_target = np.random.choice(gene_pool.keys())
            if tfbs_to_dublicate not in gene_pool[gene_target]:
                gene_pool[gene_target].append(tfbs_to_dublicate)
                flag = True
    else:
#         print "Evolution Step : " + str (s) + " TFBS Deletion"
        gene_target = np.random.choice(gene_pool.keys())
        tfbs_to_delete = np.random.choice(gene_pool[gene_target])
        gene_pool[gene_target].remove(tfbs_to_delete)
        if len(gene_pool[gene_target]) == 0:
            gene_to_delete = gene_target
            gene_pool.pop(gene_to_delete)
            for i in range(gene_to_delete, genes-1):
                gene_pool[i] = gene_pool.get(i+1)
            if not gene_to_delete == genes - 1:
                gene_pool.pop(genes - 1)
            genes = genes-1
            if genes == 0:
                print "Gene Pool Empty !"
                break


## Building coevolution network
G = nx.Graph()
for g_a in gene_pool.keys():
    for g_b in gene_pool.keys():
        if not g_a == g_b: 
            if len(set(gene_pool[g_a]).intersection(gene_pool[g_b])) > 0:
                G.add_edge(g_a, g_b)


#### Question 2


#### Question 3


#### Question 4
